# Import libraries

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

import sys
sys.path.insert(0,'/content/drive/MyDrive/')

Mounted at /content/drive/


In [ ]:
cd /content/drive/MyDrive/ROBT613_BMI_Final_Project

/content/drive/MyDrive/BMI_Final_Project


In [ ]:
!pip install mne==0.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mne: filename=mne-0.18.0-py3-none-any.whl size=6313874 sha256=3befc4a6f16e153c070aedfad700e5f73257638746fd24256a5608a64461afef
  Stored in directory: /root/.cache/pip/wheels/50/55/a4/d95e3c6ab9370b77f9c871e9882a9578a2195c64ede88694ff
Successfully built mne


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

import itertools
import pandas as pd
import pickle
import numpy as np
import mne
from sklearn.preprocessing import StandardScaler

from nu_smrutils import loaddat,subject_specific,augment_dataset,crop_data,train_model

%matplotlib inline
%load_ext autoreload
%autoreload 2
torch.manual_seed(0)

# Select device

In [ ]:
#by default
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if dev.type == 'cuda':
   print('Your GPU device name :', torch.cuda.get_device_name())

Your GPU device name : Tesla T4


# Select Dataset

## 1. BNCI2014004

In [ ]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'BNCI2014004'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


# Load subject specific data, preprocess data (filtering, data augmentation, data cropping, standard scaling), and use Pytorch dataloaders

Function for standard scaler

In [ ]:
scaler = StandardScaler()
def standard_scaling(data):
  shape1 = data.shape[1]
  shape2 = data.shape[2]
  data = data.reshape(data.shape[0],-1)
  data = scaler.fit_transform(data)
  data = data.reshape(data.shape[0],shape1,shape2)
  return data

Function for Pytorch dataloaders

In [ ]:
def get_data_loaders(dat, batch_size, EEGNET = None, dev=dev):
    # convert data dimensions to into to gray scale image format
    if EEGNET: ### EEGNet model requires the last dimension to be 1
        ff = lambda dat: torch.unsqueeze(dat, dim = -1)
    else:
        ff = lambda dat: torch.unsqueeze(dat, dim = 1)

    x_train, x_valid, x_test =  map(ff,(dat['xtrain'],dat['xvalid'],dat['xtest']))
    y_train, y_valid, y_test =  dat['ytrain'], dat['yvalid'], dat['ytest']
    print('Input data shape', x_train.shape)

    # TensorDataset & Dataloader
    train_dat = TensorDataset(x_train.to(dev), y_train.to(dev))
    val_dat   = TensorDataset(x_valid.to(dev), y_valid.to(dev))

    train_loader = DataLoader(train_dat, batch_size = batch_size, shuffle = True)
    val_loader   = DataLoader(val_dat,   batch_size = batch_size, shuffle = False)

    output = dict(dset_loaders = {'train': train_loader, 'val': val_loader},
                  dset_sizes  =  {'train': len(x_train), 'val': len(x_valid)},
                  test_data   =  {'x_test' : x_test, 'y_test' : y_test})
    return output

In [ ]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([982, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1098, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])

Data cropping
torch.Size([3120, 3, 120])
torch.Size([2946, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3120, 3, 120])
to

# ResNet50 architecture

ResNet uses residual blocks



In [ ]:
class ResBlock(nn.Module):

    def __init__(self, in_channels, inter_channels, identity_downsample=None, stride=1):
        super(ResBlock, self).__init__()
        self.expansion=4
        self.in_channels = in_channels
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=inter_channels,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(inter_channels)
        self.conv2 = nn.Conv2d(
            in_channels=inter_channels,
            out_channels=inter_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn2 = nn.BatchNorm2d(inter_channels)
        self.conv3 = nn.Conv2d(
            in_channels=inter_channels,
            out_channels=inter_channels*self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn3 = nn.BatchNorm2d(inter_channels * self.expansion)
        self.relu = nn.LeakyReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity=x.clone()
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.conv3(out)
        out = self.bn3(out)
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        out += identity
        out = self.relu(out)
        return out



class ResNet50(nn.Module):

    def __init__(self, classes=None):
        super(ResNet50, self).__init__()
        self.in_channels = 64
        self.conv1=nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(64)
        self.r=nn.LeakyReLU()
        self.mp=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.l1=self._make_layer(ResBlock, 3, inter_channels=64, stride=1)
        self.l2=self._make_layer(ResBlock, 4, inter_channels=128, stride=2)
        self.l3=self._make_layer(ResBlock, 6, inter_channels=256, stride=2)
        self.l4=self._make_layer(ResBlock, 3, inter_channels=512, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512 * 4, 100)
        self.Dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(100, classes)

    def forward(self, x):
      x = self.conv1(x)
      x = self.bn1(x)
      x = self.r(x)
      x = self.mp(x)
      x = self.l1(x)
      x = self.l2(x)
      x = self.l3(x)
      x = self.l4(x)
      x = self.avgpool(x)
      x = x.reshape(x.shape[0], -1)
      x = self.fc1(x)
      x=self.Dropout(x)
      x = self.fc2(x)

      return x

    def _make_layer(self, block, num_residual_blocks, inter_channels, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.in_channels != inter_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    inter_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(inter_channels * 4),
            )

        layers.append(
            block(self.in_channels, inter_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50
        self.in_channels = inter_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, inter_channels))

        return nn.Sequential(*layers)

# Training loop

In [ ]:
#Hyperparameter settings
num_epochs = 50
learning_rate = 1e-4
weight_decay = 1e-4
batch_size = 64
verbose = 2

#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  model = ResNet50(classes=2)

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)

  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/50
train loss: 0.0127, acc: 0.5119
val loss: 0.0128, acc: 0.5109
Epoch 2/50
train loss: 0.0109, acc: 0.5955
val loss: 0.0151, acc: 0.4746
Epoch 3/50
train loss: 0.0088, acc: 0.7000
val loss: 0.0167, acc: 0.4891
Epoch 4/50
train loss: 0.0065, acc: 0.7859
val loss: 0.0186, acc: 0.5109
Epoch 5/50
train loss: 0.0060, acc: 0.8077
val loss: 0.0211, acc: 0.4710
Epoch 6/50
train loss: 0.0048, acc: 0.8558
val loss: 0.0277, acc: 0.5254
Epoch 7/50
train loss: 0.0045, acc: 0.8641
val loss: 0.0204, acc: 0.4928
Epoch 8/50
train loss: 0.0040, acc: 0.8875
val loss: 0.0262, acc: 0.5507
Epoch 9/50
train loss: 0.0036, acc: 0.8946
val loss: 0.0330, acc: 0.5326
Epoch 10/50
train loss: 0.0043, acc: 0.8888
val loss: 0.0272, acc: 0.5181
Epoch 11/50
train loss: 0.0032, acc: 0.9135
val loss: 0.0259, acc: 0.5761
Epoch 12/50
train loss: 0.0026, acc: 0.9282
val loss: 0.0330, acc: 0.5399
Epoch 13/50
train loss: 0.0024, acc: 0.9353
val loss: 0.0384, acc: 0.5290


# Save and print results

In [ ]:
table.to_csv('results/ResNet50/BNCI2014004_resnet50.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1   0.978205  0.601449      0.42     43
Subject2   0.973863  0.563218      0.48     42
Subject3   0.951282  0.543478      0.52     13
Subject4   0.831149  0.803509      0.84      3
Subject5   0.970662  0.575439      0.56     40
Subject6   0.973397  0.623188      0.70     13
Subject7   0.560897  0.644928      0.70      1
Subject8   0.989982  0.766323      0.64     37
Subject9   0.989103  0.623188      0.50     38

Average between subjects: 0.60
Max test_score between subjects: 0.84


Repeat the same steps for other datasets
## 2. BNCI2014001

In [ ]:
itemname = 'BNCI2014001'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


In [ ]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.000001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])

Data cropping
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 

In [ ]:
#Hyperparameter settings
num_epochs = 50
learning_rate = 1e-3
weight_decay = 1e-3
batch_size = 64
verbose = 2

#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  model = ResNet50(classes=2)

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)

  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/50
train loss: 0.0234, acc: 0.5217
val loss: 0.0273, acc: 0.5135
Epoch 2/50
train loss: 0.0155, acc: 0.5016
val loss: 0.0143, acc: 0.5856
Epoch 3/50
train loss: 0.0133, acc: 0.5161
val loss: 0.0140, acc: 0.5586
Epoch 4/50
train loss: 0.0128, acc: 0.5427
val loss: 0.0130, acc: 0.4865
Epoch 5/50
train loss: 0.0146, acc: 0.5105
val loss: 0.0327, acc: 0.5135
Epoch 6/50
train loss: 0.0123, acc: 0.5282
val loss: 0.0126, acc: 0.5135
Epoch 7/50
train loss: 0.0116, acc: 0.5105
val loss: 0.0125, acc: 0.4865
Epoch 8/50
train loss: 0.0114, acc: 0.5620
val loss: 0.0127, acc: 0.5225
Epoch 9/50
train loss: 0.0108, acc: 0.6200
val loss: 0.0130, acc: 0.5225
Epoch 10/50
train loss: 0.0092, acc: 0.7101
val loss: 0.0118, acc: 0.6036
Epoch 11/50
train loss: 0.0078, acc: 0.7955
val loss: 0.0145, acc: 0.5856
Epoch 12/50
train loss: 0.0058, acc: 0.8494
val loss: 0.0139, acc: 0.6757
Epoch 13/50
train loss: 0.0047, acc: 0.8889
val loss: 0.0152, acc: 0.6036


In [ ]:
table.to_csv('results/ResNet50/BNCI2014001_resnet50.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1   0.973430  0.756757      0.70     29
Subject2   0.644928  0.585586      0.44     11
Subject3   0.987923  0.873874      0.84     21
Subject4   0.987118  0.693694      0.60     31
Subject5   0.972625  0.630631      0.44     49
Subject6   0.980676  0.684685      0.74     34
Subject7   0.495169  0.594595      0.48      4
Subject8   0.983897  0.882883      0.84     49
Subject9   1.000000  0.837838      0.82     44

Average between subjects: 0.66
Max test_score between subjects: 0.84


## 3. Weibo2014

In [ ]:
itemname = 'Weibo2014'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 10


In [ ]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Subject specific
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading subject specific data')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=standard_scaling(datt[i]['xtrain'])
  datt[i]['xvalid']=standard_scaling(datt[i]['xvalid'])
  datt[i]['xtest']=standard_scaling(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading subject specific data
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
Loading subject: 10
list index out of range

Data augmentation
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([202, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])

Data cropping
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
tor

In [ ]:
#Hyperparameter settings
num_epochs = 75
learning_rate = 1e-3
weight_decay = 1e-4
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])
sum=0
num=0
max=0

for i in range(len(datt)):
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

  model = ResNet50(classes=2)

  # optimizer and the loss function definition
  optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)
  criterion.to(dev)

  #******** Training loop *********
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:]
  y_test = dat['test_data']['y_test'][:test_samples]

  # predict test data
  preds = best_model(x_test.to(dev))
  preds_class = preds.data.max(1)[1]

  # get the accuracy
  corrects = torch.sum(preds_class == y_test.data.to(dev))
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc)
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

  table.loc[description] = tab
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/50
train loss: 0.0268, acc: 0.5174
val loss: 3.4779, acc: 0.5238
Epoch 2/50
train loss: 0.0154, acc: 0.5304
val loss: 0.6524, acc: 0.5238
Epoch 3/50
train loss: 0.0131, acc: 0.5029
val loss: 0.0902, acc: 0.4762
Epoch 4/50
train loss: 0.0128, acc: 0.5058
val loss: 0.0206, acc: 0.5238
Epoch 5/50
train loss: 0.0132, acc: 0.4899
val loss: 0.2450, acc: 0.5397
Epoch 6/50
train loss: 0.0127, acc: 0.5304
val loss: 0.1975, acc: 0.5238
Epoch 7/50
train loss: 0.0118, acc: 0.5072
val loss: 0.3702, acc: 0.5238
Epoch 8/50
train loss: 0.0127, acc: 0.5087
val loss: 0.0394, acc: 0.3968
Epoch 9/50
train loss: 0.0125, acc: 0.5116
val loss: 0.0616, acc: 0.5238
Epoch 10/50
train loss: 0.0108, acc: 0.5696
val loss: 0.2039, acc: 0.5873
Epoch 11/50
train loss: 0.0110, acc: 0.5841
val loss: 0.0599, acc: 0.5238
Epoch 12/50
train loss: 0.0104, acc: 0.5681
val loss: 0.1025, acc: 0.5238
Epoch 13/50
train loss: 0.0099, acc: 0.6333
val loss: 0.2281, acc: 0.5079


In [ ]:
table.to_csv('results/ResNet50/Weibo_resnet50.csv')

print(table)
print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

           Train_Acc   Val_Acc  Test_Acc  Epoch
Subject1    0.569565  0.587302      0.40     10
Subject2    0.952174  0.698413      0.48     31
Subject3    0.605797  0.619048      0.44     26
Subject4    0.959420  0.650794      0.44     45
Subject5    0.520290  0.714286      0.48     18
Subject6    0.737624  0.685185      0.50     29
Subject7    0.966667  0.936508      0.78     24
Subject8    0.924638  0.571429      0.46     38
Subject9    0.952174  0.682540      0.84     42
Subject10   0.695652  0.539683      0.60     29

Average between subjects: 0.54
Max test_score between subjects: 0.84
